Import Statements

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import lit,col,expr,when,coalesce
from delta.tables import DeltaTable

MERGE FUNCTIONS

In [0]:
def merge_function(df_new,name,path):
    if not DeltaTable.isDeltaTable(spark,path):
        df=df_new.withColumn("status",lit("Additional"))
        df.write.format('Delta').option("mergeSchema", "true").mode('overwrite').saveAsTable(name)
    else:
        spark.sql(f"update  {name} set status='Missing'")
        target=DeltaTable.forName(spark,f'{name}')
        target.alias("target").merge(
        source=df_new.alias("source"),
        condition="target.employee_id = source.employee_id "
        ).whenMatchedUpdate(
                    set={
                    "employee_id":"source.employee_id",
                    "employee_name" :"source.employee_name",
                    "employee_age":"source.employee_age",
                    "month1":"source.month1",
                    "status":lit("Exising")
                   }
        ).whenNotMatchedInsert(values={
                    "employee_id":"source.employee_id",
                    "employee_name" :"source.employee_name",
                   "employee_age":"source.employee_age",
                   "month1":"source.month1",
                   "status":lit("Addition")
        }).execute()
    



In [0]:
DeltaTable.isDeltaTable(spark,'dbfs:/user/hive/warehouse/hr_table')

Out[3]: True

Month -1

In [0]:
data = [
    ("dbg01", "Luffy", 20, 1),
    ("dbg02", "Zoro", 22, 1),
    ("dbg03", "Sanji", 22, 1),
]

# Define the schema with a struct type for 'month1'
schema = StructType([
    StructField("employee_id", StringType(), True),
    StructField("employee_name", StringType(), True),
    StructField("employee_age", IntegerType(), True),
    StructField("month1", IntegerType(), True)
])
df = spark.createDataFrame(data, schema)
merge_function(df,"data_source45","/dbfs/FileStore")

Month - 2

In [0]:
data = [
    ("dbg01", "Monkey D luffy", 21, 2),
    ("dbg04", "nami", 24, 2)
]
df = spark.createDataFrame(data, schema)
merge_function(df,"data_source45","dbfs:/user/hive/warehouse/data_source45")

In [0]:
%sql
select * from data_source45

employee_id,employee_name,employee_age,month1,status
dbg01,Monkey D luffy,21,2,Exising
dbg03,Sanji,22,1,Missing
dbg04,nami,24,2,Addition
dbg02,Zoro,22,1,Missing


Month - 3

In [0]:
data = [
    ("dbg02", "Ronovara zora", 23, 3),
    
]
df = spark.createDataFrame(data, schema)
merge_function(df,"data_source45","dbfs:/user/hive/warehouse/data_source45")

In [0]:
%sql
select * from data_source45

employee_id,employee_name,employee_age,month1,status
dbg01,Monkey D luffy,21,2,Missing
dbg02,Ronovara zora,23,3,Exising
dbg03,Sanji,22,1,Missing
dbg04,nami,24,2,Missing


Month - 4


In [0]:
data = [
    ("dbg01", "Monkey D luffy", 21, 4),
    ("dbg03", "prince sanji", 23, 4),
    ("dbg05", "ussopu", 19, 4),
]
df = spark.createDataFrame(data, schema)
merge_function(df,"data_source45","dbfs:/user/hive/warehouse/data_source45")

In [0]:
%sql
select * from data_source45

employee_id,employee_name,employee_age,month1,status
dbg01,Monkey D luffy,21,4,Exising
dbg02,Ronovara zora,23,3,Missing
dbg03,prince sanji,23,4,Exising
dbg05,ussopu,19,4,Addition
dbg04,nami,24,2,Missing
